In [1]:
import pandas
csvFile = pandas.read_csv('cleaned100k_data1.csv', sep='|', encoding='utf-8', on_bad_lines='skip')

csvFile.shape

csvFile["id"] = csvFile.index + 1


/tmp/ipykernel_26553/78617517.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas
/tmp/ipykernel_26553/78617517.py:2: DtypeWarning: Columns (19,27,32,33,38,39,58,59,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  csvFile = pandas.read_csv('cleaned100k_data1.csv', sep='|', encoding='utf-8', on_bad_lines='skip')


In [2]:
%pip install pyodbc
%pip install duckdb
%pip install splink
%pip install usaddress
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyodbc
import os
import pandas as pd
import re
import usaddress
import time
import phonetics


In [4]:
csvFile.columns

Index(['SynchronizationTimestamp', 'Status', 'PER_SiteCode', 'PER_RowID',
       'PER_RootID', 'PER_ClientID', 'PER_CreateDate', 'PER_UpdateDate',
       'PER_IsPatient', 'PER_IsContact', 'PER_IsFamilyMember', 'PER_IsPrimary',
       'PER_IsVersion', 'PER_LastName', 'PER_FirstName', 'PER_MiddleName',
       'PER_ThirdName', 'PER_FourthName', 'PER_NameSuffix', 'PER_NamePrefix',
       'PER_SSN', 'PER_DOB', 'PER_PrimaryLanguage', 'PER_Age', 'PER_Ethnicity',
       'PER_ReportedRace', 'PER_DateOfUSArrival', 'PER_Sex',
       'PER_MaritalStatus', 'PER_StateNumber', 'PER_ResidenceCounty',
       'PER_CountryofResidence', 'PER_CountryofBirth',
       'PER_PrimaryNationality', 'PER_ParentOrGuardianName',
       'PER_LastNameAlphaUp', 'PER_FirstNameAlphaUp', 'PER_DOD',
       'PER_DeceasedStatus', 'PER_StatusFlag', 'PER_StreetAddress',
       'PER_Apartment', 'PER_City', 'PER_State', 'PER_Zip', 'PER_CensusTract',
       'PER_Latitude', 'PER_Longitude', 'PER_AddressStandardized',
       'PER_Co

In [5]:
cleaned_df9_subset=csvFile[['PER_LastName', 'PER_FirstName', 'PER_MiddleName',  'PER_SSN', 'PER_DOB',  'PER_HomePhone', 'PER_CellularPhoneOrPager',
        'PER_Email', 'PER_Apartment', 'PER_City', 'PER_State', 'PER_Zip', 'id']]

all_columns = list(cleaned_df9_subset) # Creates list of all column headers
cleaned_df9_subset[all_columns] = cleaned_df9_subset[all_columns].astype(str)

/tmp/ipykernel_26553/3890102094.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df9_subset[all_columns] = cleaned_df9_subset[all_columns].astype(str)


In [6]:
#Feature engineering for RecordLinkage/Deduplication

import pandas as pd
import phonetics

# Define a function to apply the dmetaphone phonetic algorithm to each name in the column
def dmetaphone_name(name):
    if name is None:
        pass
    else:
        return phonetics.dmetaphone(name)

# Apply the function to the "first_name", lastname and middlename columns using the apply method
cleaned_df9_subset['firstname_dm'] = cleaned_df9_subset['PER_FirstName'].apply(dmetaphone_name)
cleaned_df9_subset['lastname_dm'] = cleaned_df9_subset['PER_LastName'].apply(dmetaphone_name)
cleaned_df9_subset['middlename_dm'] = cleaned_df9_subset['PER_MiddleName'].apply(dmetaphone_name)


# Create a new column "full_name" by combining "first_name" and "surname" columns
cleaned_df9_subset['full_name'] = cleaned_df9_subset['PER_FirstName'] + ' ' + cleaned_df9_subset['PER_LastName']

/tmp/ipykernel_26553/2709407767.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df9_subset['firstname_dm'] = cleaned_df9_subset['PER_FirstName'].apply(dmetaphone_name)
/tmp/ipykernel_26553/2709407767.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df9_subset['lastname_dm'] = cleaned_df9_subset['PER_LastName'].apply(dmetaphone_name)
/tmp/ipykernel_26553/2709407767.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [7]:
cleaned_df9_subset['PER_DOB']=  pd.to_datetime(cleaned_df9_subset['PER_DOB'])

/tmp/ipykernel_26553/86631419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df9_subset['PER_DOB']=  pd.to_datetime(cleaned_df9_subset['PER_DOB'])


In [8]:
counts = cleaned_df9_subset.isna().sum()
print(counts.sort_values())

PER_LastName                  0
lastname_dm                   0
firstname_dm                  0
id                            0
PER_Zip                       0
PER_State                     0
PER_City                      0
middlename_dm                 0
PER_Apartment                 0
PER_CellularPhoneOrPager      0
PER_HomePhone                 0
PER_SSN                       0
PER_MiddleName                0
PER_FirstName                 0
PER_Email                     0
full_name                     0
PER_DOB                     139
dtype: int64


In [9]:
from splink.duckdb.linker import DuckDBLinker
#from splink.spark.linker import SparkLinker

settings = {
    "unique_id_column_name": "id",
    "link_type": "dedupe_only",
}

linker = DuckDBLinker(cleaned_df9_subset, settings)

In [10]:
from splink.duckdb.blocking_rule_library import block_on

blocking_rules = [
        block_on("lastname_dm"),
        block_on("PER_LastName"),
        block_on("full_name"),
        block_on("firstname_dm"),
        block_on("PER_FirstName"),
        block_on("PER_DOB"),
        block_on("PER_HomePhone"),
        block_on("PER_Email"),
        block_on("PER_CellularPhoneOrPager"),
        block_on("middlename_dm"),
        block_on("PER_SSN")
]
linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules)

alt.Chart(...)

In [11]:
from splink.duckdb.blocking_rule_library import block_on

blocking_rules = [
        block_on("lastname_dm"),
        block_on("PER_LastName"),
        block_on("full_name"),
        block_on("firstname_dm"),
        block_on("PER_FirstName"),
        block_on("PER_DOB"),
        block_on("PER_HomePhone"),
        #block_on("PER_Email"),
        #block_on("PER_CellularPhoneOrPager"),
        block_on("middlename_dm"),
        block_on("PER_SSN")
]
linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules)

alt.Chart(...)

In [11]:
from splink.duckdb.blocking_rule_library import block_on

blocking_rules = [
        block_on("PER_LastName"),
        block_on("full_name"),
        block_on("PER_FirstName"),
        block_on("PER_DOB"),
]
linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules)

alt.Chart(...)

In [12]:
import splink.duckdb.comparison_template_library as ctl
from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl


import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl


settings = {
    "unique_id_column_name": "id",
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": blocking_rules,
    "comparisons": [
        cl.exact_match("firstname_dm"),
        cl.exact_match("lastname_dm"),
        ctl.name_comparison("full_name"),
        ctl.email_comparison("PER_Email", include_username_fuzzy_level=False),
        cl.levenshtein_at_thresholds("PER_SSN", [2]),
        ctl.date_comparison("PER_DOB"),
    ],
    "retain_intermediate_calculation_columns": True
}

linker = DuckDBLinker(cleaned_df9_subset, settings, set_up_basic_logging=False)
deterministic_rules = [
    "l.PER_SSN = r.PER_SSN and l.PER_DOB = r.PER_DOB",
    "l.PER_FirstName = r.PER_FirstName and l.PER_DOB = r.PER_DOB",
    "l.PER_LastName = r.PER_LastName and l.PER_DOB = r.PER_DOB",
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

Probability two random records match is estimated to be  6.51e-05.
This means that amongst all possible pairwise record comparisons, one in 15,362.95 are expected to match.  With 4,999,950,000 total possible comparisons, we expect a total of around 325,455.00 matching pairs


In [13]:
linker.estimate_u_using_random_sampling(max_pairs=1e8)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - firstname_dm (no m values are trained).
    - lastname_dm (no m values are trained).
    - full_name (no m values are trained).
    - PER_Email (no m values are trained).
    - PER_SSN (no m values are trained).
    - PER_DOB (no m values are trained).


In [14]:
from numpy import fix
training_blocking_rule = block_on("PER_DOB")
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."PER_DOB" = r."PER_DOB"

Parameter estimates will be made for the following comparison(s):
    - firstname_dm
    - lastname_dm
    - full_name
    - PER_Email
    - PER_SSN

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - PER_DOB

Level Levenshtein <= 2 on comparison PER_SSN not observed in dataset, unable to train m value

Iteration 1: Largest change in params was 0.263 in the m_probability of PER_Email, level `All other comparisons`
Iteration 2: Largest change in params was -0.31 in the m_probability of full_name, level `Exact match full_name`
Iteration 3: Largest change in params was 0.0774 in the m_probability of lastname_dm, level `All other comparisons`
Iteration 4: Largest change in params was 0.0232 in the m_probability of full_name, level `All other comparisons`
Iteration 5: Largest change in params was 0

In [15]:
training_blocking_rule = block_on(["firstname_dm", "lastname_dm"])
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."firstname_dm" = r."firstname_dm") AND (l."lastname_dm" = r."lastname_dm")

Parameter estimates will be made for the following comparison(s):
    - full_name
    - PER_Email
    - PER_SSN
    - PER_DOB

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - firstname_dm
    - lastname_dm

Level Levenshtein <= 2 on comparison PER_SSN not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.909 in the m_probability of PER_DOB, level `Exact match`
Iteration 2: Largest change in params was 0.0172 in the m_probability of PER_DOB, level `All other comparisons`
Iteration 3: Largest change in params was 0.00861 in the m_probability of full_name, level `All other comparisons`
Iteration 4: Largest change in params was 0.00819 in the m_probability of full_name, level `All other comparisons`
Ite

In [16]:
training_blocking_rule = block_on(["PER_FirstName"])
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."PER_FirstName" = r."PER_FirstName"

Parameter estimates will be made for the following comparison(s):
    - firstname_dm
    - lastname_dm
    - full_name
    - PER_Email
    - PER_SSN
    - PER_DOB

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 

Level All other comparisons on comparison firstname_dm not observed in dataset, unable to train m value

Level Levenshtein <= 2 on comparison PER_SSN not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.762 in the m_probability of lastname_dm, level `Exact match`
Iteration 2: Largest change in params was 0.453 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.152 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.00414 in probability_two_random_records_match
Iteration 

In [17]:
training_blocking_rule = block_on(["PER_Zip"])
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."PER_Zip" = r."PER_Zip"

Parameter estimates will be made for the following comparison(s):
    - firstname_dm
    - lastname_dm
    - full_name
    - PER_Email
    - PER_SSN
    - PER_DOB

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 

Iteration 1: Largest change in params was 0.63 in the m_probability of full_name, level `Jaro_winkler_similarity >= 0.8`
Iteration 2: Largest change in params was 0.0697 in the m_probability of firstname_dm, level `All other comparisons`
Iteration 3: Largest change in params was -0.0427 in the m_probability of firstname_dm, level `Exact match`
Iteration 4: Largest change in params was 0.0136 in the m_probability of firstname_dm, level `All other comparisons`
Iteration 5: Largest change in params was -0.00424 in the m_probability of firstname_dm, level `Exact match`
Iteration 6: Largest c

In [18]:
linker.match_weights_chart()

alt.VConcatChart(...)

In [19]:
linker.m_u_parameters_chart()

alt.HConcatChart(...)

In [20]:
linker.unlinkables_chart()

alt.LayerChart(...)

In [21]:
settings = linker.save_model_to_json("saved_model1.json", overwrite=True)

In [22]:
df_predictions = linker.predict(threshold_match_probability=0.2)

In [23]:
clusters = linker.cluster_pairwise_predictions_at_threshold(df_predictions, threshold_match_probability=0.5)
df_predictions.as_pandas_dataframe()
df_predictions

Completed iteration 1, root rows count 49066
Completed iteration 2, root rows count 43848
Completed iteration 3, root rows count 42737
Completed iteration 4, root rows count 42022
Completed iteration 5, root rows count 36726
Completed iteration 6, root rows count 26757
Completed iteration 7, root rows count 20290
Completed iteration 8, root rows count 12519
Completed iteration 9, root rows count 5924
Completed iteration 10, root rows count 2304
Completed iteration 11, root rows count 1080
Completed iteration 12, root rows count 447
Completed iteration 13, root rows count 228
Completed iteration 14, root rows count 188
Completed iteration 15, root rows count 84
Completed iteration 16, root rows count 43
Completed iteration 17, root rows count 53
Completed iteration 18, root rows count 34
Completed iteration 19, root rows count 21
Completed iteration 20, root rows count 26
Completed iteration 21, root rows count 8
Completed iteration 22, root rows count 2
Completed iteration 23, root row

Table name in database: `__splink__df_predict_90ae95b4f`

To retrieve records, you can call the following methods on this object:
`.as_record_dict(limit=5)` or `.as_pandas_dataframe(limit=5)`.

You may omit the `limit` argument to return all records.

This table represents the following splink entity: __splink__df_predict

In [24]:
.as_pandas_dataframe()

SyntaxError: invalid syntax (851758953.py, line 1)

In [25]:
df_predictions

Table name in database: `__splink__df_predict_90ae95b4f`

To retrieve records, you can call the following methods on this object:
`.as_record_dict(limit=5)` or `.as_pandas_dataframe(limit=5)`.

You may omit the `limit` argument to return all records.

This table represents the following splink entity: __splink__df_predict

In [26]:
df_model_output=df_predictions.as_pandas_dataframe()

In [27]:
df_model_output.shape

(3216913, 33)

In [28]:
df_model_output.columns

Index(['match_weight', 'match_probability', 'id_l', 'id_r', 'firstname_dm_l',
       'firstname_dm_r', 'gamma_firstname_dm', 'bf_firstname_dm',
       'lastname_dm_l', 'lastname_dm_r', 'gamma_lastname_dm', 'bf_lastname_dm',
       'full_name_l', 'full_name_r', 'gamma_full_name', 'bf_full_name',
       'PER_Email_l', 'PER_Email_r', 'gamma_PER_Email', 'bf_PER_Email',
       'PER_SSN_l', 'PER_SSN_r', 'gamma_PER_SSN', 'bf_PER_SSN', 'PER_DOB_l',
       'PER_DOB_r', 'gamma_PER_DOB', 'bf_PER_DOB', 'PER_FirstName_l',
       'PER_FirstName_r', 'PER_LastName_l', 'PER_LastName_r', 'match_key'],
      dtype='object')

In [30]:
max_number = max(df_model_output['id_r'])
print(max_number)

99999


In [35]:
len(df_model_output.query('id_l == id_r'))

0

In [36]:
len(df_model_output.query('id_l != id_r'))

3216913

In [ ]:
#So according to this model and Tinas model, there are no duplicates